In [1]:
import torch
from torch import nn

from torch.optim import lr_scheduler

from torch.utils.data import random_split,Dataset,DataLoader

import torch.nn.functional as F
import torch.nn.init as init

from momentumnet import MomentumNet
from momentumnet import transform_to_momentumnet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
from joblib.externals.loky.backend.context import get_context

import time
import copy
import random
import pickle
import tarfile

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Dataset Class

In [2]:
class MyDataset(Dataset):
    def __init__(self,gene_matrix,cell_type):
        
        self.gene_matrix = torch.from_numpy(gene_matrix).float()
        self.cell_type = torch.from_numpy(cell_type).squeeze(1)
    

    def __len__(self):
        
        return self.gene_matrix.shape[0]

    def __getitem__(self,idx):
        
        data = (self.gene_matrix[idx],self.cell_type[idx])
        
        return data

# Create PBMC DataLoader

In [3]:
def create_pbmc1_loader(pbmc_path,cell_type_path,batch_size=128):

    pbmc = pd.read_csv(pbmc_path,header=None)
    cell_type = pd.read_csv(cell_type_path,index_col = 0)
    
    full_dataset = MyDataset(pbmc.values,cell_type.values)

    #Random split(0.7,0.3)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    # Define DataLoaders
    # use 'loky' to work with joblib
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,   batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader


# MLP

## Initial

In [4]:
def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear):
        init.normal(m.weight)

## MLP Model

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        
        self.lr1 = nn.Linear(3500,1000)
        self.lr2 = nn.Linear(1000,128)
        self.lr3 = nn.Linear(128,64)
        
        self.lr = nn.Linear(64,9)
        
        self.apply(_weights_init)
        
    def forward(self,x):
        x = F.relu(self.lr1(x))
        x = self.lr2(x)
        x = F.relu(self.lr3(x))
        
        output = self.lr(x)
        
        return output

# SGD_Train

In [6]:
def SGD_training(network, SGD_steps, lr, momentum, data_loader):

    network.to(device)
    network.train()

    # Create optimizer and criterion
    criterion = nn.CrossEntropyLoss(reduction='mean')
    
    optimizer = torch.optim.SGD(network.parameters(), lr=lr, momentum=momentum, nesterov=True, weight_decay=0.0001)
    
    total_step = len(data_loader)
    
    for s in range(0, SGD_steps):
        total_loss = 0
        for i, (genes, types) in enumerate(data_loader): 
            
            genes = genes.to(device)
            types = types.to(device)
            
            # Forward pass
            outputs = network(genes) 
            
            loss = criterion(outputs, types)
            total_loss += loss.item()
            # Backward and optimize
            optimizer.zero_grad()
            
            loss.backward()
            
            #Gradient Value Clipping
            nn.utils.clip_grad_value_(network.parameters(), clip_value=1.0)
            
            optimizer.step()
            
            del loss, outputs
            
            #if (i+1) % 100 == 0:
        total_loss = total_loss / len(data_loader.sampler)
        print ("Epoch [{}/{}], Loss: {}".format(s+1, SGD_steps, total_loss))
        
            
            
    
    # move network back to cpu and return
    network.cpu()
    
    return network

# GA

In [7]:
def crossover_and_mutation(parents, sigma=0.01):

    
    base_sd = parents[0].state_dict()
    keys = base_sd                    # use all layers to be affected
    
    # Sum of the weights of the parent
    for i in range(1, len(parents)):
        parent_sd = parents[i].state_dict()
        for key in keys:
            base_sd[key] = base_sd[key] + parent_sd[key]
            
    
    # Average and add mutation
    num_parents = len(parents)
    
    for key in keys:
        
        tensor_size = base_sd[key].size()
        random_tensor = torch.normal(mean=0.0, std=sigma, size=tensor_size)
        
        base_sd[key] = (base_sd[key] / num_parents) + random_tensor
    
    # create offspring
    offspring = MLP()
    
    offspring.load_state_dict(base_sd)
    
    return offspring
    

def create_offspring(population, fitness, rho, sigma):

    
    # Perform selection
    parents = random.choices(population, weights=fitness, k=rho) 
    
    # Perform crossover and mutation
    offspring = crossover_and_mutation(parents, sigma)
    
    
    return offspring

def sigmoid(x):
    
    return 1/(1 + np.exp(-x))


def GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, data_loader):
    
    #Calculate fitness of trained population

    fitness = [calc_loss(population[i], data_loader) for i in range(pop_size)]
    
    print(f"--- -- Finished fitness evaluation, length: {len(fitness)}")
    
    #Create offspring population
    fitness_weighted = [sigmoid(-f) for f in fitness]   # take inverse of loss so lower losses get higher fitness-values
    offspring_population = [create_offspring(population, fitness_weighted, rho, sigma) for i in range(offspring_size)]
    print("--- -- Finished creating offspring population")
    
    #Evaluate fitness of offsprings 
    
    offspring_fitness = [calc_loss(offspring_population[i], data_loader) for i in range(offspring_size)]
    print("--- -- Finished evaluating fitness of offspring population")
    
    # Combine fitness and population lists
    
    combined_fitness = fitness + offspring_fitness
    combined_population = population + offspring_population
    
    # sort and select population by their fitness values
    
    sorted_population = [pop for _, pop in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    sorted_fitness = [loss for loss, _ in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    
    m = int(pop_size * elitist_level)
    new_population = sorted_population[0:m]
    
    # Fill up rest of population
    difference = pop_size - m
    remaining_population = list(set(sorted_population) - set(new_population))
    filler_population = random.sample(remaining_population, difference)
    
    # assemble new population and return
    new_population = new_population + filler_population
    
    return new_population, sorted_fitness

In [8]:
def calc_loss(network, data_loader):
    
    network.to(device)
    criterion = nn.CrossEntropyLoss(reduction='mean')
    total_loss = 0.0
    
    for i, (genes, types) in enumerate(data_loader):
        
        genes = genes.to(device)
        types = types.to(device)
        
        # Forward pass
        outputs = network(genes)
        loss = criterion(outputs, types)
        
        total_loss += loss
        
        del loss, outputs
        
    network.cpu()
    
    return float(total_loss/len(data_loader.sampler))

# GA_Neural

In [9]:
def GA_Neural_train(population,
                    pop_size,
                    max_generations, 
                    SGD_steps, GA_steps, 
                    offspring_size, elitist_level, rho,
                    learning_rate,
                    train_loader):
    
    print(f"Starting with population of size: {pop_size}")
    
    
    for k in range(max_generations):
        print(f"Currently in generation {k+1}")
        
        #SGD
        print(f"--- Starting SGD")
        
        # Sequential version
        population = [SGD_training(population[i], SGD_steps, learning_rate, 0.9, train_loader) for i in range(pop_size)]
        
        print(f"--- Finished SGD")
         
        # GA
        print(f"--- Starting GA")
        GA_start = time.time()
        sorted_fitness = []          # store the sorted fitness values to maybe use in data collection
        for i in range(0, GA_steps):
            
            sigma = 0.01 / (k+1)
            population, sorted_fitness = GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, train_loader)
        
        GA_end = time.time()
        print(f"--- Finished GA,Time:{(GA_start-GA_end)*1000}ms")
        
        
    print(f"Finished training process")
    return population

# Prepare training process

In [10]:
# Hyperparameters
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
batch_size = 128
pop_size = 10
max_generations = 10
SGD_steps = 10
GA_steps = 1
offspring_size = 10
elitist_level = 0.6
rho = 2
learning_rate = 1e-5
pbmc_1 = "data/pbmc_1_pca.csv"
cell_type_pbmc1 = "data/cell_type_pbmc1.csv"
pbmc_2 = "data/pbmc_2_pca.csv"
cell_type_pbmc2 = "data/cell_type_pbmc2.csv"

In [11]:
train_loader, test_loader = create_pbmc1_loader(pbmc_1,cell_type_pbmc1,batch_size = batch_size)

In [12]:
# Create population and start training process
population = [MLP() for i in range(pop_size)]

# Train Process

In [13]:
Train_start = time.time()
trained_population = GA_Neural_train(population=population,
                                    pop_size = pop_size,
                                    max_generations=max_generations,
                                    SGD_steps=SGD_steps,GA_steps=GA_steps,
                                    offspring_size=offspring_size,elitist_level=elitist_level,rho=rho,
                                    learning_rate=learning_rate,
                                    train_loader=train_loader)
Train_end = time.time()
print(f"All Time:{(Train_start-Train_end)*1000}ms")

Starting with population of size: 10
Currently in generation 1
--- Starting SGD
Epoch [1/10], Loss: 2352.5521173469388
Epoch [2/10], Loss: 1912.7696237244897
Epoch [3/10], Loss: 1507.373801020408
Epoch [4/10], Loss: 1160.9634375
Epoch [5/10], Loss: 889.5119419642857
Epoch [6/10], Loss: 726.309706632653
Epoch [7/10], Loss: 556.8213727678572
Epoch [8/10], Loss: 472.5841772959184
Epoch [9/10], Loss: 425.2756393494898
Epoch [10/10], Loss: 365.99777981505105
Epoch [1/10], Loss: 3578.156173469388
Epoch [2/10], Loss: 3026.742704081633
Epoch [3/10], Loss: 2408.5448788265307
Epoch [4/10], Loss: 1938.471855867347
Epoch [5/10], Loss: 1554.6905739795918
Epoch [6/10], Loss: 1283.8523883928572
Epoch [7/10], Loss: 1057.9760427295919
Epoch [8/10], Loss: 922.7232079081633
Epoch [9/10], Loss: 807.4747672193878
Epoch [10/10], Loss: 709.5285395408164
Epoch [1/10], Loss: 1554.323488520408
Epoch [2/10], Loss: 1157.216517857143
Epoch [3/10], Loss: 882.9627232142857
Epoch [4/10], Loss: 719.1156648596939
Epoch

Epoch [2/10], Loss: 175.2808171237245
Epoch [3/10], Loss: 172.28901586415816
Epoch [4/10], Loss: 160.01234135841835
Epoch [5/10], Loss: 154.73566764987245
Epoch [6/10], Loss: 169.7193757971939
Epoch [7/10], Loss: 134.01684829400511
Epoch [8/10], Loss: 129.78714564732144
Epoch [9/10], Loss: 123.84270169005102
Epoch [10/10], Loss: 114.73400091677296
Epoch [1/10], Loss: 188.71060467155613
Epoch [2/10], Loss: 174.10698022959184
Epoch [3/10], Loss: 164.28180564413265
Epoch [4/10], Loss: 154.49636599170918
Epoch [5/10], Loss: 148.76082071109693
Epoch [6/10], Loss: 146.39143973214286
Epoch [7/10], Loss: 135.1673457429847
Epoch [8/10], Loss: 125.8467483458227
Epoch [9/10], Loss: 120.04146404655613
Epoch [10/10], Loss: 126.36954918686224
Epoch [1/10], Loss: 183.64194116709183
Epoch [2/10], Loss: 181.44494539221938
Epoch [3/10], Loss: 175.60741948341837
Epoch [4/10], Loss: 157.8919511320153
Epoch [5/10], Loss: 154.2981536989796
Epoch [6/10], Loss: 147.38128228635205
Epoch [7/10], Loss: 136.12434

Epoch [2/10], Loss: 67.51668267697704
Epoch [3/10], Loss: 63.146435048628824
Epoch [4/10], Loss: 61.050667948820156
Epoch [5/10], Loss: 57.30264219945791
Epoch [6/10], Loss: 56.95464823820153
Epoch [7/10], Loss: 52.57516232860332
Epoch [8/10], Loss: 54.41299615353954
Epoch [9/10], Loss: 46.23752967055963
Epoch [10/10], Loss: 44.15678137954401
Epoch [1/10], Loss: 69.29210259885204
Epoch [2/10], Loss: 66.34381566884566
Epoch [3/10], Loss: 68.31248485331632
Epoch [4/10], Loss: 60.12675841039541
Epoch [5/10], Loss: 57.94799176897322
Epoch [6/10], Loss: 58.51577108577806
Epoch [7/10], Loss: 52.59778180803571
Epoch [8/10], Loss: 48.141126335299745
Epoch [9/10], Loss: 49.51461485570791
Epoch [10/10], Loss: 43.37898804022341
Epoch [1/10], Loss: 71.7807057158801
Epoch [2/10], Loss: 64.98152812101404
Epoch [3/10], Loss: 64.95622349330357
Epoch [4/10], Loss: 60.12137057557398
Epoch [5/10], Loss: 59.15645438058036
Epoch [6/10], Loss: 53.49172029456314
Epoch [7/10], Loss: 54.33925562021684
Epoch [8

Epoch [2/10], Loss: 23.28964502762775
Epoch [3/10], Loss: 22.186336268210898
Epoch [4/10], Loss: 20.824990458585777
Epoch [5/10], Loss: 20.63969051438935
Epoch [6/10], Loss: 19.703189971301022
Epoch [7/10], Loss: 18.036583027742346
Epoch [8/10], Loss: 17.866920315489477
Epoch [9/10], Loss: 15.310372211689852
Epoch [10/10], Loss: 15.320153646663744
Epoch [1/10], Loss: 25.123505859375
Epoch [2/10], Loss: 23.443641008649553
Epoch [3/10], Loss: 23.611203712930486
Epoch [4/10], Loss: 23.459549759845345
Epoch [5/10], Loss: 20.687004992426658
Epoch [6/10], Loss: 18.55307357243129
Epoch [7/10], Loss: 29.645156785614635
Epoch [8/10], Loss: 16.29171613420759
Epoch [9/10], Loss: 16.059227469308034
Epoch [10/10], Loss: 14.840071797273596
Epoch [1/10], Loss: 25.397003921197385
Epoch [2/10], Loss: 24.15816650390625
Epoch [3/10], Loss: 22.206704998405613
Epoch [4/10], Loss: 21.096788853236607
Epoch [5/10], Loss: 19.60678216428173
Epoch [6/10], Loss: 18.51608682437819
Epoch [7/10], Loss: 18.6169257712

Epoch [2/10], Loss: 11.155728731739277
Epoch [3/10], Loss: 6.410877996950734
Epoch [4/10], Loss: 6.194620573082749
Epoch [5/10], Loss: 5.514236046148806
Epoch [6/10], Loss: 5.133996871636838
Epoch [7/10], Loss: 4.838401554652623
Epoch [8/10], Loss: 4.5606962585449216
Epoch [9/10], Loss: 4.283848599025181
Epoch [10/10], Loss: 4.069265485491071
Epoch [1/10], Loss: 7.30009840284075
Epoch [2/10], Loss: 6.923058023258132
Epoch [3/10], Loss: 6.404900995371293
Epoch [4/10], Loss: 6.009157492189991
Epoch [5/10], Loss: 5.595021362304688
Epoch [6/10], Loss: 5.284010721323441
Epoch [7/10], Loss: 5.140494967090841
Epoch [8/10], Loss: 4.639870138362962
Epoch [9/10], Loss: 4.3609044164540816
Epoch [10/10], Loss: 4.14811378323302
Epoch [1/10], Loss: 7.744237807916135
Epoch [2/10], Loss: 7.00525074553733
Epoch [3/10], Loss: 6.5736045993104275
Epoch [4/10], Loss: 6.239397243577606
Epoch [5/10], Loss: 5.656867021443892
Epoch [6/10], Loss: 5.3180358388472575
Epoch [7/10], Loss: 5.0268803872867505
Epoch [

# Test Model

In [14]:
# Test the model
def test_function(network, data_loader):
    # init accuracy
    accuracy = 0.0
    
    network.to(device)
    network.eval()
    
    with torch.no_grad():
        correct = 0
        total = 0
        for genes, types in data_loader:
            genes = genes.to(device)
            types = types.to(device)
            outputs = network(genes)
            _, predicted = torch.max(outputs.data, 1)
            
            total += genes.size(0)
            
            correct += (predicted == types).sum().item()

        accuracy =  correct / total
        #print('Accuracy of the model on the test images: {} %'.format(accuracy))
        
    # send network back to cpu
    network.cpu()
    
    return accuracy

# Test

In [15]:
test_accuracy = [test_function(trained_population[i],test_loader) for i in range(pop_size)]

In [16]:
test_accuracy

[0.6895238095238095,
 0.6895238095238095,
 0.6876190476190476,
 0.6885714285714286,
 0.6866666666666666,
 0.6847619047619048,
 0.6914285714285714,
 0.6866666666666666,
 0.6876190476190476,
 0.6876190476190476]

In [17]:
np.array(test_accuracy).mean()

0.6880000000000001